In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import spatial

#prepare data with generic scores

txt1 = pd.read_csv('scores/train.txt', sep=" ",  header = None)
txt2 = pd.read_csv('scores/test.txt', sep=" ",  header = None)
txt3 = pd.read_csv('scores/val.txt', sep=" ", header = None)
data = pd.concat([txt1,txt2,txt3])
data.columns = ['image', 'score']
data.image  = data.image.str.replace('/home/ubuntu/crop/','FaceNet_512/')
data['image'] =data['image'].str.replace(".jpg", ".csv")
data['image'] =data['image'].str.replace(".png", ".csv")
data = data.sort_values(by=['score'])
n_training, acc = average_face_method(data,'score',test = 0.3)

print(n_training, acc)

/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:15: FutureWarning: The default value of regex will change from True to False in a future version.


1627 0.727015558698727


In [1]:
#prepare the personal score data

data = pd.read_excel('scores/date_scores_all.xlsx')
data = data.drop(['image','mean_gen'], axis=1)
data['path'] =data['path'].str.replace('/home/ubuntu/ME-beautydatabase/images','FaceNet_512')
data['path'] =data['path'].str.replace(".jpg", ".csv")
data['path'] =data['path'].str.replace(".png", ".csv")
data = data[data['path'].notna()]
data = data.rename(columns={"path": "image"})
df_acc = pd.DataFrame(columns = ['user', 'img','accuracy'])
for column in data:
    
    if column != "image":
       
       #print(column)
       df = data[['image',column]]
       df = df.dropna() 
        
       df_male = df[df['image'].str.contains("/male")]
       df_female = df[df['image'].str.contains("/female")]
        
       if len(df_male) > 0: 
    
          pr1 = len(df_male.loc[df_male[column] == 1])/len(df_male)
       else:
          pr1=0
    
        
       if len(df_female) > 0: 
    
          pr2 = len(df_female.loc[df_female[column] == 1])/len(df_female)
       else:
          pr2=0
    
    
       if pr1 > 0.9:
          df= df_female  
          print('remove males')
       if pr2 > 0.9:
          df= df_male  
          print('remove females') 
        
       df_final = pd.DataFrame(columns = ['image', column])
        
       for index, row in df.iterrows():
            
           try:
             #print(row)
             v1 = np.loadtxt(row['image'], delimiter=',')
             df_final.loc[len(df_final)] = [row['image'],row[column]]   
           except:
             #print('no')
             pass
        
       train_number=100   
    
       if len(df_final)>train_number + 30:
          n_training, acc = average_face_method(df_final, column, train_number=100)
          df_acc.loc[len(df_acc)] = [column, n_training, acc]
       else:
          pass     
        
print(df_acc.mean())
print(df_acc)
       #print(len(train),(r[0,0]+r[1,1])/r.sum()) 
    

NameError: name 'pd' is not defined

In [80]:
df = df_acc.loc[df_acc['accuracy']==0]
df

,user,img,accuracy


In [18]:
def average_face(train, attr = True):
   
   #create a vector with average attr or not face

   df0 = pd.DataFrame()
   df1 = pd.DataFrame()
   v = [0] * 512
   ind = 0

   train2 = train.loc[train['True'] == attr]

   for index, row in train2.iterrows():
    
      row['image'] = row['image'].replace(".jpg", ".csv")
      row['image'] = row['image'].replace(".png", ".csv")

      try:

         v1 = np.loadtxt(row['image'], delimiter=',')
         v=np.add (v,v1)
         ind += 1
        
      except:
        
         pass
    

   avrg = v/ind

   return avrg, ind

def average_face_method(df,column, train_number = 100):
    
       df['True'] = df[column].apply(lambda x: True if (x > df[column].mean()) else False)
       df = df.sample(frac=1).reset_index(drop=True)
       train= df[:train_number]
       test= df[train_number:]

       attr,x1 = average_face(train, True)
       not_attr,x2 = average_face(train, False)
        
       result = pd.DataFrame(columns=['pred', 'real'])
       test = test.sample(frac=1).reset_index(drop=True)
     
       for index, row in test.iterrows():
          #print(row['image'])
          try:
        
             v1 = np.loadtxt(row['image'], delimiter=',')
             d1 = spatial.distance.euclidean(v1, attr)
             d2 = spatial.distance.euclidean(v1, not_attr)
     
             #print(d1,d2)
             if d1>=d2: 
            
                result.loc[len(result)]=[False,row['True']]    
                
             else: 

                result.loc[len(result)]=[True,row['True']]
            
    
          except:
             pass
       #print(result)
       r = result.groupby(["pred", "real"]).size()

       n_train_images = x1+x2
       
       try:
          acc = (r[0,0]+r[1,1])/r.sum() 
       except:
          acc = 0
       
       return n_train_images, acc